In [4]:
data = '''
t = triangle A B C
'''

lexer = SolverLexer()
for tok in lexer.tokenize(data):
    print(tok)

Token(type='SYMBOL', value='t', lineno=2, index=1)
Token(type='ASSIGN', value='=', lineno=2, index=3)
Token(type='TRIANGLE', value='triangle', lineno=2, index=5)
Token(type='SYMBOL', value='A', lineno=2, index=14)
Token(type='SYMBOL', value='B', lineno=2, index=16)
Token(type='SYMBOL', value='C', lineno=2, index=18)


In [5]:
from lark import Lark

l = Lark('''start: WORD "," WORD "!"

            %import common.WORD   // imports from terminal library
            %ignore " "           // Disregard spaces in text
         ''')

print( l.parse("Hello, World!") )

Tree(start, [Token(WORD, 'Hello'), Token(WORD, 'World')])


In [6]:
solver_grammar = """
    start: _NL? problem
    problem: ([expr | decl] _NL)+
    decl: symbol "=" expr
    expr: binoperation
        | method
        | symbol
        | literal
    method: symbol "(" parameter ")"
    parameter: expr ("," expr)*
    binoperation: expr OPERATOR expr
    symbol: SYMBOL
    literal: LITERAL
    
    SYMBOL: /[a-zA-Z][a-zA-Z0-9]*/
    LITERAL: /\d*\.?\d+/
    OPERATOR: /[\/*\-+]/
    
    %import common.INT -> NUMBER
    %import common.NEWLINE -> _NL
    %import common.WS_INLINE
    
    %ignore WS_INLINE
"""

In [7]:
parser = Lark(solver_grammar)
sample = """
t = triangle(a, b, c)
perimeter(t, 5)
eq(dist(a, b), 2)
eq(dist(b, c), 1)
solve(dist(a, c))
"""
print(parser.parse(sample).pretty())
print(parser.parse(sample))
str(parser.parse(sample).children[0].children[0].children[0].children[0])

start
  problem
    decl
      symbol	t
      expr
        method
          symbol	triangle
          parameter
            expr
              symbol	a
            expr
              symbol	b
            expr
              symbol	c
    expr
      method
        symbol	perimeter
        parameter
          expr
            symbol	t
          expr
            literal	5
    expr
      method
        symbol	eq
        parameter
          expr
            method
              symbol	dist
              parameter
                expr
                  symbol	a
                expr
                  symbol	b
          expr
            literal	2
    expr
      method
        symbol	eq
        parameter
          expr
            method
              symbol	dist
              parameter
                expr
                  symbol	b
                expr
                  symbol	c
          expr
            literal	1
    expr
      method
        symbol	solve
        parameter
          expr
    

't'

In [8]:
from problem import *
from lark import Tree, Token

p = Problem() 

methods = {
    'triangle': p.triangle,
    'perimeter': p.perimeter,
    'eq': p.eq,
    'dist': p.dist,
    'solve': p.solve
}

def process_expression(root, problem):
    if root.data == 'literal' or root.data == 'symbol':
        return(str(root.children[0]))
    elif root.data == 'binoperation':
        return(visit_branch(root, problem))
    elif root.data == 'method':
        method = visit_branch(root, problem)
        #
        return(methods.get(method[0], lambda: 'Invalid')(*method[1]))

    #return visit_branch(root)

def visit_branch(root, problem):
    if isinstance(root, Tree):
        params = [visit_branch(child, problem) for child in root.children]
       # print(params)
        if root.data == 'expr':
            return process_expression(root.children[0], problem)
        elif root.data == 'decl':
            problem.bind(*params)
        elif root.data == 'literal' or root.data == 'symbol':
            return params[0]
        return params
    if isinstance(root, Token):
        return str(root)
               
visit_branch(parser.parse(sample), p)

[[['t', <problem.Triangle at 0x2b0d7c4ba90>], None, None, None, 2.0]]

In [170]:
    if root.data == 'symbol':
        print(str(root.children[0]))
        return str(root.children[0])
    elif root.data == 'literal':
        return str(root.children[0])
    elif root.data == 'binoperation':
        children = root.children
        print(visit_branch(children[0], problem))
        #print([visit_branch(children[0], problem), str(root.children[1]), visit_branch(children[2], problem)])
    else:

SyntaxError: unexpected EOF while parsing (<ipython-input-170-08a24cef08fd>, line 10)